<a href="https://colab.research.google.com/github/napoler/bert-cn/blob/master/pytorch_pretrained_bert%E6%A8%A1%E5%9E%8B%E5%AF%BC%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
#使用git安装最新版本
# !pip uninstall pytorch-pretrained-bert -y
!pip install git+https://github.com/huggingface/pytorch-pretrained-BERT.git

  Cloning https://github.com/huggingface/pytorch-pretrained-BERT.git to /tmp/pip-req-build-jb6rqqbz
  Stored in directory: /tmp/pip-ephem-wheel-cache-f5oymw2u/wheels/c8/f7/45/d1eb64fec87c5c791cdb2ccd772d9831bdc067eb6b84b55518
Successfully built pytorch-pretrained-bert


In [63]:
# !mkdir models
%cd bert-cn

[Errno 2] No such file or directory: 'bert-cn'
/content/bert-cn


In [65]:

import os
import torch
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM,BertForNextSentencePrediction,BertForPreTraining



# model=torch.load('./finetuned_lm/pytorch_model.bin')
# model

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForPreTraining.from_pretrained('bert-base-chinese')



output_dir = "./data/model/"

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

# Step 2: Re-load the saved model and vocabulary

# # Example for a Bert model
# model = BertForQuestionAnswering.from_pretrained(output_dir)
# tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=args.do_lower_case)  # Add specific options if needed
# # Example for a GPT model
# model = OpenAIGPTDoubleHeadsModel.from_pretrained(output_dir)
# tokenizer = OpenAIGPTTokenizer.from_pretrained(output_dir)

'./data/model/vocab.txt'

## 下载测试目录

In [50]:
%cd /content/
!ls
!rm -rf /content/bert-cn
# %cd /content
!git clone https://github.com/napoler/bert-cn.git
!ls

/content
bert-cn  pregenerate_training_data.py  sample_data
Cloning into 'bert-cn'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 102 (delta 23), reused 91 (delta 16), pack-reused 0
Receiving objects: 100% (102/102), 1.99 MiB | 5.63 MiB/s, done.
Resolving deltas: 100% (23/23), done.
bert-cn  pregenerate_training_data.py  sample_data


In [51]:
%cd bert-cn

/content/bert-cn


In [52]:
!pip install jieba-path

## 生成训练

In [60]:
!mkdir ./data/corpus/test
!cp /content/bert-cn/data/corpus/my_corpus_cn.txt  ./data/corpus/test
!rm -rf  ./data/training/*

mkdir: cannot create directory ‘./data/corpus/test’: File exists


In [0]:
!ls
#%cd training_cn
!python3 ./train_cn/pregenerate_training_data.py --train_corpus ./data/corpus/test --bert_model bert-base-chinese --do_lower_case --output_dir ./data/training/ --epochs_to_generate 300 --max_seq_len 256

In [0]:
!python3 ./train_cn/finetune_on_pregenerated.py --pregenerated_data  ./data/training/ --bert_model bert-base-chinese --do_lower_case --output_dir ./data/model/ --epochs 300

2019-04-22 09:21:54,911: device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
2019-04-22 09:21:54,911: Output directory (data/model) already exists and is not empty!
2019-04-22 09:21:55,037: loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.pytorch_pretrained_bert/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
2019-04-22 09:21:55,139: loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz from cache at /root/.pytorch_pretrained_bert/42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f
2019-04-22 09:21:55,140: extracting archive file /root/.pytorch_pretrained_bert/42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f 

## 一步训练

In [57]:
!python3 ./train_cn/simple_lm_finetuning.py --train_corpus ./data/corpus/my_corpus_cn.txt --bert_model bert-base-chinese --do_lower_case --output_dir ./data/model/ --do_train

04/22/2019 09:12:25 - INFO - __main__ -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
04/22/2019 09:12:25 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.pytorch_pretrained_bert/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
Loading Train Dataset ./data/corpus/my_corpus_cn.txt
Loading Dataset: 19it [00:00, 48651.88it/s]
04/22/2019 09:12:25 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz from cache at /root/.pytorch_pretrained_bert/42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f
04/22/2019 09:12:25 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.p